In [ ]:
import tifffile
import numpy as np
import os
import pandas as pd
from skimage.morphology import binary_erosion, skeletonize, label
from skimage import draw
from skan import Skeleton, summarize
import scipy.ndimage as ndi
from scipy.sparse.csgraph import shortest_path
from skan.csr import skeleton_to_csgraph
from pathlib import Path
from scipy.ndimage import distance_transform_edt
from skimage.segmentation import watershed
import pickle

In [ ]:
input_path = Path(r"sample_test_Data/Input")
output_path = r"sample_test_Data/Output"

## Auxilliary Functions

In [ ]:
def create_temp_dir(directory):
    '''
    Create a temporary directory if it does not exist and clear it if it does.
    '''
    # Check if the directory already exists
    if not os.path.exists(directory):
        # Create the directory
        os.makedirs(directory)
        print(f"Directory '{directory}' created successfully.")
    else:
        print(f"Directory '{directory}' already exists.")
        print(f"Clearing the directory...")
        # Clearing the directory for the next run
        for filename in os.listdir(directory):
            file_path = os.path.join(directory, filename)
            try:
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.unlink(file_path)
                elif os.path.isdir(file_path):
                    directory.rmtree(file_path)
            except Exception as e:
                print('Failed to delete %s. Reason: %s' % (file_path, e))

def create_dir(directory):
    '''
    Create a temporary directory if it does not exist and clear it if it does.
    '''
    # Check if the directory already exists
    if not os.path.exists(directory):
        # Create the directory
        os.makedirs(directory)
        print(f"Directory '{directory}' created successfully.")
    else:
        print(f"Directory '{directory}' already exists.")

In [ ]:
#Function for the computation of the number of bifurcations and end points
def branching_points(branch_data):
    branch_points=0
    end_points=0
    node_origin = branch_data['node-id-src'].values.tolist()
    node_destination = branch_data['node-id-dst'].values.tolist()
    all_nodes = node_origin + node_destination
    aux_list = np.array(all_nodes)
    bpoints_id = []
    epoints_id = []
    for element in np.unique(aux_list):
        occur = all_nodes.count(element)
        if occur>=3:
            branch_points=branch_points+1
            bpoints_id.append(element)
        elif occur==1:
            end_points=end_points+1
            epoints_id.append(element)
    return branch_points, bpoints_id, end_points, epoints_id


def radius_3d_main(skeleton, branch_data, mask, boundary):
    pixel_graph, coordinates_ = skeleton_to_csgraph(skeleton, spacing=[1,1,1])
    dist_matrix, predecessors = shortest_path(pixel_graph, directed=True, indices=branch_data['node-id-src'].to_numpy(), return_predecessors=True)
    #dist_matrix has size (#sourceids as in len(branch_data['node-id-src']), #all nodes in the skeleton)
    
    ## iterate through each branch and check the direction
    all_major = []
    all_minor = []
    all_radii = []
    all_paths = []
    avg_branch_radii = []
    avg_branch_minor_axis = []
    avg_branch_major_axis = []
    all_directions_units = []


    for i in range(len(branch_data)):
        
        #node indices (i is the node-id-src, because it was used before to compute dist_matrix and predecessors)
        b =  int(branch_data.iloc[i]['node-id-dst']) 

        # Check if there is a path between the two nodes (a and b)
        if np.isinf(dist_matrix[i, b]):
            print("No path exists between node a and node b.")
            all_paths.append([])
            avg_branch_radii.append(0)
            continue
        else:
            # Reconstruct the path from a to b
            path = [(coordinates_[0][b], coordinates_[1][b], coordinates_[2][b])]
            b = predecessors[i, b]
            while b >= 0:
                path.insert(0, (coordinates_[0][b], coordinates_[1][b], coordinates_[2][b]))
                b = predecessors[i, b]

            path = np.asarray(path)

            #compute the direction of the branch
            delta_x = (branch_data.iloc[i]['image-coord-src-0'])-(branch_data.iloc[i]['image-coord-dst-0'])
            delta_y = (branch_data.iloc[i]['image-coord-src-1'])-(branch_data.iloc[i]['image-coord-dst-1'])
            delta_z = (branch_data.iloc[i]['image-coord-src-2'])-(branch_data.iloc[i]['image-coord-dst-2'])

            direction_unit = np.asarray([delta_x, delta_y, delta_z])
            direction_unit = direction_unit / np.linalg.norm(direction_unit)
            all_directions_units.append(direction_unit)
            
            major_axes, minor_axes, radii = compute_radii_aux(path, mask, boundary, direction_unit)
            # print(f"Radii {i}: {radii}")
            all_major = all_major + major_axes
            all_minor = all_minor + minor_axes
            all_radii = all_radii + radii
            all_paths.append(path)

            if radii:
                avg_branch_radii.append(np.mean(radii))
                avg_branch_minor_axis.append(np.mean(minor_axes))
                avg_branch_major_axis.append(np.mean(major_axes))
            else:
                avg_branch_radii.append(0)
                avg_branch_minor_axis.append(0)
                avg_branch_major_axis.append(0)
                
    return (np.asarray(all_major), np.asarray(all_minor), np.asarray(all_radii), all_paths,
            avg_branch_radii, avg_branch_minor_axis, avg_branch_major_axis, all_directions_units
            )

def extract_2d_slice(segmentation_mask, boundary, point, direction_unit, radius=20):
    D = np.dot(direction_unit, point)
    min_point = np.maximum(point - radius, [0, 0, 0])
    max_point = np.minimum(point + radius, segmentation_mask.shape)
    
    grid_x, grid_y, grid_z = np.meshgrid(
        np.arange(min_point[0], max_point[0]),
        np.arange(min_point[1], max_point[1]),
        np.arange(min_point[2], max_point[2]),
        indexing='ij')
    
    voxel_centers = np.column_stack((grid_x.ravel(), grid_y.ravel(), grid_z.ravel()))
    distances = np.abs(np.dot(voxel_centers, direction_unit) - D)
    plane = (distances < 0.5).reshape(grid_x.shape)
    
    out_mask = np.zeros(plane.shape)
    out_mask = np.logical_and(plane, segmentation_mask[min_point[0]:max_point[0], min_point[1]:max_point[1], min_point[2]:max_point[2]])
    
    out_boundary = np.zeros(boundary.shape)
    out_boundary = np.logical_and(plane, boundary[min_point[0]:max_point[0], min_point[1]:max_point[1], min_point[2]:max_point[2]])
    
    new_point = point * (min_point == 0) + radius * (min_point != 0)

    return out_boundary, out_mask, new_point

def compute_radii_aux(path, mask, boundary, direction_unit):
    major_axes = []
    minor_axes = []
    radii = []
    tam_ = np.shape(path)[0]
    first_point = True
    for p in range(int(tam_/2), tam_):

        point = np.asarray(path[p])

        aux_boundary, aux_mask, point = extract_2d_slice(mask, boundary, point, direction_unit)

        aux_mask = label(aux_mask)
        
        l = aux_mask[point[0], point[1], point[2]]
        
        aux_boundary[aux_mask!=l] = 0
        
        indices_ = np.argwhere(aux_boundary) # get indices of the contour

        if np.shape(indices_)[0]>0:

            all_distances = np.sqrt(np.sum((indices_ - point)**2, axis=-1)) #Euclidean distance
            #from the point to each point in the boundary

            major_curr = np.max(all_distances)
            minor_curr = np.min(all_distances)
            
            if first_point:
                major_axes.append(major_curr)
                minor_axes.append(minor_curr)
                radii.append(np.mean(all_distances))
                first_point = False
            else:
                delta_radius_major = np.abs(major_curr-major_axes[-1])
                delta_radius_minor = np.abs(minor_curr-minor_axes[-1])
                if delta_radius_major<4:
                    major_axes.append(major_curr)
                    minor_axes.append(minor_curr)
                    radii.append(np.mean(all_distances))
                else:
                    break
            
    path = np.flip(path,0)
    for p in range(int(tam_/2), tam_):

        point = np.asarray(path[p])

        aux_boundary, aux_mask, point = extract_2d_slice(mask, boundary, point, direction_unit)

        aux_mask = label(aux_mask)
        
        l = aux_mask[point[0], point[1], point[2]]
        
        aux_boundary[aux_mask!=l] = 0
        
        indices_ = np.argwhere(aux_boundary) # get indices of the contour

        if np.shape(indices_)[0]>0:

            all_distances = np.sqrt(np.sum((indices_ - point)**2, axis=-1)) #Euclidean distance
            #from the point to each point in the boundary
            
            major_curr = np.max(all_distances)
            minor_curr = np.min(all_distances)
            
            if first_point:
                major_axes.append(major_curr)
                minor_axes.append(minor_curr)
                radii.append(np.mean(all_distances))
                first_point = False
            else:
                delta_radius_major = np.abs(major_curr-major_axes[-1])
                delta_radius_minor = np.abs(minor_curr-minor_axes[-1])
                if delta_radius_major<4:
                    major_axes.append(major_curr)
                    minor_axes.append(minor_curr)
                    radii.append(np.mean(all_distances))
                else:
                    break

    return major_axes, minor_axes, radii


def skeleton_pruning(skeleton: Skeleton, spacing = [1,1,1]) -> Skeleton:
    """Prune a skeleton using recursion removing the specified branch type.
    Parameters
    ----------
    remove_branch_type: int
        Remove branches of the specified type options are the types returned under `branch-type` by summarize and
    the default is `1` which removes junction-to-endpoint branches.
          0 endpoint-to-endpoint (isolated branch)
          1 junction-to-endpoint
          2 juntciont-to-junction
          3 isolated cycle
    Returns
    -------
    Skeleton
        Returns a new Skeleton instance.
    """

    original = Skeleton(skeleton, spacing=spacing)
    branch_data = summarize(original, separator='-')

    print(f"Before Pruning: {len(original.paths.indices)}")

    # pruned = pruned.prune_paths(branch_data.loc[(branch_data["branch-type"] == 2) & (branch_data['branch-distance'] < 5.0)].index)
    # branch_data = summarize(pruned, separator='-')

    # print(f"After Pruning: {len(pruned.paths.indices)}")

    pruned = original.prune_paths(branch_data.loc[branch_data["branch-type"] == 3].index) # Pruning the loops
    branch_data = summarize(pruned, separator='-')

    print(f"After Loop Pruning Iter1 : {len(pruned.paths.indices)}")

    pruned = pruned.prune_paths(branch_data.loc[branch_data["branch-type"] == 3].index) # Pruning the loops
    branch_data = summarize(pruned, separator='-')

    print(f"After Loop Pruning Iter2: {len(pruned.paths.indices)}")

    pruned = pruned.prune_paths(branch_data.loc[(branch_data["branch-type"] == 0) & (branch_data['branch-distance'] < 12.0)].index)
    branch_data = summarize(pruned, separator='-')

    print(f"After Isolated Branch Pruning: {len(pruned.paths.indices)}")

    pruned = pruned.prune_paths(branch_data.loc[(branch_data["branch-type"] == 1) & (branch_data['branch-distance'] < 12.0)].index)
    branch_data = summarize(pruned, separator='-')

    print(f"After Small Endpoint Pruning Iter1 : {len(pruned.paths.indices)}")

    pruned = pruned.prune_paths(branch_data.loc[(branch_data["branch-type"] == 1) & (branch_data['branch-distance'] < 12.0)].index)
    branch_data = summarize(pruned, separator='-')
    print(f"After Small Endpoint Pruning Iter2: {len(pruned.paths.indices)}")


    pruned = pruned.prune_paths(branch_data.loc[branch_data["branch-type"] == 3].index) # Pruning the loops
    branch_data = summarize(pruned, separator='-')

    print(f"After Loop Pruning Iter3: {len(pruned.paths.indices)}")

    pruned = Skeleton(pruned.skeleton_image, spacing=[1,1,1])

    return pruned

def extract_dapi_values_from_arrays(path_array, matrix):
    values = []
    for coord in path_array:
        if len(coord) == 3:
            x, y, z = coord[0], coord[1], coord[2]
            if 0 <= x < matrix.shape[0] and 0 <= y < matrix.shape[1] and 0 <= z < matrix.shape[2]:
                values.append(int(matrix[x, y, z]))
    return values

def get_corresponding_dapi_path(cell_result_path):
    try:
        # Convert to Path object for easier manipulation
        path = Path(cell_result_path)
        
        # Get the series folder (parent of the results folder)
        series_folder = path.parent
        
        # Construct the DAPI results path
        dapi_path = series_folder / "DAPI_results"
        
        # Verify the DAPI path exists
        if dapi_path.exists():
            return str(dapi_path)
        else:
            print(f"Warning: No DAPI_results folder found for {cell_result_path}")
            return None
            
    except Exception as e:
        print(f"Error processing path {cell_result_path}: {str(e)}")
        return None
    
def calculate_branch_ratios(branch_data):
    # Filter branches of type 1
    type_1_branches = branch_data[branch_data['branch-type'] == 1]

    # Initialize new columns for branch ratios with NaN
    branch_data['Branch 1 Ratio'] = np.nan
    branch_data['Branch 2 Ratio'] = np.nan
    branch_data['Protrusion-Factor-1'] = np.nan
    branch_data['Protrusion-Factor-2'] = np.nan


    # Process only type 1 branches
    for index, branch in type_1_branches.iterrows():
        source_node = branch['node-id-src']
        destination_node = branch['node-id-dst']
        current_radius = branch['Radius']
        current_eu_dist = branch['euclidean-distance']

        # Find all related branches (excluding the current branch)
        related_branches = branch_data[
            ((branch_data['node-id-src'] == source_node) | (branch_data['node-id-dst'] == source_node) |
             (branch_data['node-id-src'] == destination_node) | (branch_data['node-id-dst'] == destination_node)) &
            (branch_data.index != index)
        ]

        # Sort and pick the first two branches
        related_branches = related_branches.head(2)

        # Calculate the ratios for the first two branches if they exist
        if len(related_branches) > 0:
            branch_1_radius = related_branches.iloc[0]['Radius']
            branch_1_ratio = branch_1_radius / current_radius
            protrusion_factor = branch_1_radius / current_eu_dist
            branch_data.at[index, 'Branch-1-Ratio'] = branch_1_ratio
            branch_data.at[index, 'Protrusion-Factor-1'] = protrusion_factor

            if len(related_branches) > 1:
                branch_2_radius = related_branches.iloc[1]['Radius']
                branch_2_ratio = branch_2_radius / current_radius
                protrusion_factor = branch_2_radius / current_eu_dist
                branch_data.at[index, 'Branch-2-Ratio'] = branch_2_ratio        
                branch_data.at[index, 'Protrusion-Factor-2'] = protrusion_factor

    return branch_data

In [ ]:
def create_branch_instance_mask(skeleton, mask, branch_data, paths_final):
    """
    Creates instance segmentation using watershed from skeleton to vessel boundaries
    """
    # Initialize seeds mask with unique labels for each branch
    seeds = np.zeros_like(skeleton, dtype=np.uint16)
    branch_to_label = {}
    
    # Create seeds for each branch
    for idx, path in enumerate(paths_final, start=1):
        if len(path) > 0:
            path = np.array(path)
            seeds[path[:, 0], path[:, 1], path[:, 2]] = idx
            branch_to_label[idx-1] = idx
    
    # Calculate distance transform from vessel boundaries
    # We invert the mask because distance_transform_edt works on background
    dist = distance_transform_edt(mask)
    
    # Use watershed to grow regions from seeds
    # The -dist means regions grow faster where they are further from boundaries
    instance_mask = watershed(-dist, seeds, mask=mask)
    
    return instance_mask, branch_to_label

def create_branch_points_mask(skeleton, branch_data, bpoints_id):
    """
    Creates a mask highlighting branch points
    """
    branch_points_mask = np.zeros_like(skeleton, dtype=np.uint8)
    
    for bp_id in bpoints_id:
        branches_with_point = branch_data[
            (branch_data['node-id-src'] == bp_id) | 
            (branch_data['node-id-dst'] == bp_id)
        ].iloc[0]
        
        if branches_with_point['node-id-src'] == bp_id:
            x = int(branches_with_point['image-coord-src-0'])
            y = int(branches_with_point['image-coord-src-1'])
            z = int(branches_with_point['image-coord-src-2'])
        else:
            x = int(branches_with_point['image-coord-dst-0'])
            y = int(branches_with_point['image-coord-dst-1'])
            z = int(branches_with_point['image-coord-dst-2'])
        
        # Just mark the point and its immediate neighbors
        branch_points_mask[x, y, z] = 1
    
    return branch_points_mask

def save_branch_visualization(skeleton, mask, branch_data, paths_final, outputdir, input_filename):
    """
    Saves branch instance mask and branch point mask
    """
    print("Creating branch instance mask...")
    branch_mask, branch_to_label = create_branch_instance_mask(skeleton, mask, branch_data, paths_final)
    branch_mask = np.swapaxes(branch_mask, 0, 2)
    
    print("Creating branch points mask...")
    _, bpoints_id, _, _ = branching_points(branch_data)
    branch_points_mask = create_branch_points_mask(skeleton, branch_data, bpoints_id)
    branch_points_mask = np.swapaxes(branch_points_mask, 0, 2)
    
    print(f"Saving visualizations to {outputdir}...")
    tifffile.imwrite(f'{outputdir}/{input_filename}_branch_labels_nnunet.tif', branch_mask)
    tifffile.imwrite(f'{outputdir}/{input_filename}_branch_points_nnunet.tif', branch_points_mask)
    print("Visualization files saved successfully!")
    
    return branch_to_label

def get_group_name(input_path):
    """
    Extracts the group name from the input path
    """

    # Split the filepath into blocks using '/' as the delimiter
    blocks = input_path.split('/')

    # Find the block containing the word 'acquisition' (case-insensitive)
    for block in blocks:
        if 'acquisition' in block.lower():
            group_name = block
            break
            
    group_name = "_".join(group_name.split('_')[:-2])
    return group_name

## Get all valid isotropic Blood Vessel Images

In [ ]:
def process_icam2_paths(input_dir, output_dir):
    """
    Process directories to find Icam2 images and create corresponding result folders.
    
    Args:
        input_dir (str): Path to the input directory containing processed images
        output_dir (str): Path to create DAPI results folders
        
    Returns:
        Tuple[List[str], List[str]]: Lists of (input DAPI paths, output result folder paths)
    """

    
    # Initialize lists to store paths
    icam2_paths = []
    dapi_paths = []
    icam2_result_paths = []
    
    # Convert to Path objects for easier handling
    input_path = Path(input_dir)
    output_path = Path(output_dir)
    
    # Create output directory if it doesn't exist
    output_path.mkdir(parents=True, exist_ok=True)
    
    # Walk through all directories and subdirectories
    for root, dirs, files in os.walk(input_path):

        
        # Convert current root to Path object
        root_path = Path(root)
        
        # Check if we're in an isotropic_image folder
        if root_path.name == "isotropic_image":
            file_name = "C1-Icam2-Blood-Vessels.tif"
            # dapi_file_name = "C4-DAPI-XZ.tif"

            # Look for C4-DAPI-XZ.tif in files
            if file_name in files:
                # Get the full path to the DAPI XZ image
                full_path = root_path / file_name
                # adpi_path = root_path / dapi_file_name
                
                # Get the series folder name (parent of isotropic_image)
                series_folder = root_path.parent.name
                
                # Create corresponding output folder structure
                result_folder = output_path / series_folder / "ICAM2_results"
                result_folder.mkdir(parents=True, exist_ok=True)
                
                # Add paths to lists
                icam2_paths.append(str(full_path))
                # dapi_path = root_path / dapi_file_name
                icam2_result_paths.append(str(result_folder))
                
                print(f"Found  ICAM2 image: {full_path}")
                # print(f"Found DAPI image: {dapi_path}")
                print(f"Created results folder: {result_folder}")
    
    return icam2_paths, icam2_result_paths

In [ ]:
bv_paths, result_paths = process_icam2_paths(input_path, output_path)

In [ ]:
for bv_input_file, result_output_directory in zip(bv_paths, result_paths):
    print(f"Executing Blood Vessel Pipeline for :{bv_input_file}")
    # File paths
    input_file = bv_input_file
    input_filename = os.path.basename(input_file).split('.')[0]
    input_dir = os.path.dirname(input_file)
    output_dir = result_output_directory

    # Get DAPI Segmentation
    DAPI_path_parent = get_corresponding_dapi_path(result_output_directory)
    DAPI_path = os.path.join(DAPI_path_parent, "C4-DAPI-XZ_reconstructed_cleaned_xy.tif")
    print(f"Dapi Path found: {DAPI_path}")
    dapi = tifffile.imread(DAPI_path)
    dapi = np.swapaxes(dapi, 0, 2)

    # Change here - when changing the input name of the segmentation output
    msk_name = f'{input_filename}_nnunet_reconstructed_cleaned_filtered.tif'

    # Default values for the other columns - CHANGE THIS WHEN THE ISOTROPIC/NON-ISOTROPIC
    default_values = {
        "resx": 1,
        "resy": 1,
        "resz": 1,
    }

   
    ellipsoid = draw.ellipsoid(3,3,2, spacing=(1,1,1), levelset=False)
    ellipsoid = ellipsoid.astype('uint8')
    ellipsoid = ellipsoid[1:-1,1:-1,1:-1]

    vessel_features = pd.DataFrame(columns=('Image', 'Group', 'Branching Points Density', 
                                    'Mean Branch Length', 'Mean Vessel Radius', 'Mean Minor Axis Length',
                                    'Mean Major Axis Length'))

    print('Mask Name: {}'.format(msk_name))

    #get the resolution information
    dimx = default_values["resx"] 
    dimy = default_values["resy"]
    dimz = default_values["resz"]
    print('dimensions: {} {} {}'.format(dimx, dimy, dimz))

    #mask and region of interest (ROI)
    print("here",os.path.join(output_dir, msk_name))
    mask = tifffile.imread(os.path.join(output_dir, msk_name)) # To be tested
    mask[mask!=0] = 1
    mask = mask.astype('uint8')
    mask = np.swapaxes(mask, 0, 2)
    print('Mask Shape: {}'.format(np.shape(mask)))

    
    mask_roi = (mask*1).astype('uint8')
    print('skeletonization')
    skeleton = skeletonize(mask_roi)
    skeleton = skeleton.astype('uint8')

    #compute the vascular density and avascular area
    total_area = len(mask) #total area of the ROI
    # vasc_dens = (len(mask[mask==1]) / (total_area) ) * 100 #vascular density
    # avas_area = (len(mask[mask==0]) / (total_area) ) * 100 #avascular area

    #Adding Skeleton Cleaning 
    pruned_skeleton = skeleton_pruning(skeleton, spacing=[1,1,1])
    branch_data = summarize(pruned_skeleton,separator='-')
    bpoints, bids, epoints, eids = branching_points(branch_data)

    print('Skeletons Features Computed')
    mask = (mask*1).astype('uint8')

    ellipsoid = draw.ellipsoid(1,1,1, spacing=(1,1,1), levelset=False)
    ellipsoid = ellipsoid.astype('uint8')
    er = binary_erosion(mask, ellipsoid)
    boundaries = mask - er

    radius_func_results = radius_3d_main(pruned_skeleton.skeleton_image, branch_data, mask, boundaries)
    major_axes_final = radius_func_results[0]
    minor_axes_final = radius_func_results[1]
    radii_final = radius_func_results[2]
    paths_final = radius_func_results[3]
    branch_radii = radius_func_results[4]
    branch_minor_axis = radius_func_results[5]
    branch_major_axis = radius_func_results[6]
    direction_unit = radius_func_results[7]

    branch_data["Radius"] = branch_radii
    branch_data["Minor-Axis"] = branch_minor_axis
    branch_data["Major-Axis"] = branch_major_axis

    branch_data['Direction-Unit'] = direction_unit
    branch_data['Direction-Unit-X'] = branch_data['Direction-Unit'].apply(lambda x: x[0].item())
    branch_data['Direction-Unit-Y'] = branch_data['Direction-Unit'].apply(lambda x: x[1].item())
    branch_data['Direction-Unit-Z'] = branch_data['Direction-Unit'].apply(lambda x: x[2].item())
    branch_data = branch_data.drop(columns=['Direction-Unit'])
    
    print("Saving branch viz")
    branch_to_label = save_branch_visualization(pruned_skeleton, mask, branch_data, paths_final, output_dir, input_filename)
    
    # Add the branch label IDs to branch_data before saving
    branch_data['Branch-Label-ID'] = branch_data.index.map(lambda x: branch_to_label.get(x, -1))
    branch_data = calculate_branch_ratios(branch_data)

    #Saving the numpy arrays
    np.save(f'{output_dir}/{input_filename}_major_axes_nnunet.npy', major_axes_final)
    np.save(f'{output_dir}/{input_filename}_minor_axes_nnunet.npy', minor_axes_final)
    np.save(f'{output_dir}/{input_filename}_radii_nnunet.npy', radii_final)

    # Store results in a DataFrame for the current mask
    vessel_features = pd.DataFrame({
        'Image': [msk_name],
        'Group': get_group_name(input_file),
        'Branching-Points-Density': [bpoints / total_area],
        'Mean-Branch-Length': [branch_data['branch-distance'].mean()],
        'Mean-Vessel-Radius': [np.mean(radii_final)],
        'Mean-Minor-Axis-Length': [np.mean(minor_axes_final)],
        'Mean-Major-Axis-Length': [np.mean(major_axes_final)]
    })

    # Create csv file with image name and features
    result_file = f'{output_dir}/{input_filename}_features_nnunet.csv'
    skeleton_result_file = f'{output_dir}/{input_filename}_skeleton_features_nnunet.csv'

    vessel_features.to_csv(result_file, sep=',', index=False)
    print(f"Saved sample feature statistics csv for {msk_name} to {result_file}")

    branch_data['Tortuosity-Index'] = (branch_data['branch-distance'] - branch_data['euclidean-distance']) / branch_data['branch-distance']
    branch_data['Path'] = paths_final
    branch_data['No-of-paths'] = (
        branch_data.groupby(['node-id-src', 'node-id-dst'])['branch-distance']
        .rank(method='dense', ascending=True).astype(int)
    )
    branch_data['DAPI-Values'] = branch_data['Path'].apply(lambda path_array: extract_dapi_values_from_arrays(path_array, dapi))
    branch_data['Dapi'] = branch_data['DAPI-Values'].apply(
        lambda values: max(set(values), key=values.count) if values else None
    )
    branch_data = branch_data.drop(columns=['DAPI-Values'])
    branch_data = branch_data.drop(columns=['Path'])

    branch_data.to_csv(skeleton_result_file, sep=',', index=False)
    print(f"Saved Skeleton branch results for {msk_name} to {skeleton_result_file}")
    
    all_paths = [pruned_skeleton.path_coordinates(i) for i in range(pruned_skeleton.n_paths)]
    result_file_all_paths = f'{output_dir}/{input_filename}_all_paths_nnunet.pkl'
    pickle.dump(all_paths, open(result_file_all_paths, 'wb'))
    print(f"Saved Path array for {msk_name} to {result_file_all_paths}")
    
print("Processing complete !")